In [ ]:
# MOUNT TO DRIVE MANUALLY (see 1M_replication.ipynb)
mbs = 2048  # set to maximum that fits on GPU
device = "cpu"  # device where translator is run
step_size = 1000  # set to roughly mbs

In [1]:
!pip install datasets
!pip install ctranslate2
!pip install OpenNMT-py==2.*
!pip install sentencepiece
import ctranslate2
import sentencepiece as spm
from datasets import load_dataset
from tqdm.auto import tqdm
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.6/226.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [2]:
!wget https://s3.amazonaws.com/opennmt-models/transformer-ende-wmt-pyOnmt.tar.gz
!tar xf transformer-ende-wmt-pyOnmt.tar.gz
!ct2-opennmt-py-converter --model_path averaged-10-epoch.pt --output_dir ende_ctranslate2
!apt install libcublas11

--2024-03-12 15:31:08--  https://s3.amazonaws.com/opennmt-models/transformer-ende-wmt-pyOnmt.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.52.48, 52.217.126.128, 52.216.144.173, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.52.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 362915089 (346M) [application/x-gzip]
Saving to: ‘transformer-ende-wmt-pyOnmt.tar.gz’

transformer-ende-wm 100%[===================>] 346.10M  53.0MB/s    in 6.7s    

2024-03-12 15:31:15 (51.9 MB/s) - ‘transformer-ende-wmt-pyOnmt.tar.gz’ saved [362915089/362915089]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcublaslt11
The following NEW packages will be installed:
  libcublas11 libcublaslt11
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 226 MB of archives.
After this operation, 498 MB of additional disk space will b

In [3]:
trn_set = load_dataset("roneneldan/TinyStories", split="train")
val_set = load_dataset("roneneldan/TinyStories", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [4]:
os.makedirs("/content/drive/My Drive/TinyStoriesTranslate", exist_ok=True)
translator = ctranslate2.Translator("ende_ctranslate2/", device=device)
sp = spm.SentencePieceProcessor("sentencepiece.model")

In [5]:
def translate_ts(dataset, dset_name, start_ind, end_ind, mbs=2048, verbose=True):
  if verbose: print("Preprocessing")
  big_text = ""
  for story in dataset[start_ind:end_ind]["text"]:
    big_text += story
    big_text += " "
  sentences = [s + "." for s in big_text.split(".")][:-1]
  if verbose: print("Encoding")
  encodings = sp.encode(sentences, out_type=str)
  if verbose: print("Translating")
  encodings_tra = [tr.hypotheses[0] for tr in translator.translate_batch(encodings, beam_size=1, max_batch_size=mbs)]
  if verbose: print("Decoding")
  translations = sp.decode(encodings_tra)
  if verbose: print("Saving")
  with open(f"/content/drive/My Drive/TinyStoriesTranslate/{dset_name}_de_{start_ind}_{end_ind}.csv", "w") as f:
    for t in translations:
      f.write(t + " ")
  if verbose: print("Done")

In [ ]:
for i in tqdm(range(0, len(trn_set)+1, step_size)):
  translate_ts(trn_set, "ts_train", i, i+step_size, mbs=mbs, verbose=False)

  0%|          | 0/21198 [00:00<?, ?it/s]

In [6]:
for i in tqdm(range(0, len(val_set)+1, step_size)):
  translate_ts(val_set, "ts_eval", i, i+step_size, mbs=mbs, verbose=False)